In [1]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import matplotlib.pyplot as plt
import pickle
from functional.backtester import Backtester as b

In [2]:
start_date = datetime(2020,1,1)
end_date = datetime(2023,1,1)

In [3]:
market = Market()
speculation_db = ADatabase("forex")
classification_db = ADatabase("forex_classification")
experimental = ADatabase("forex_experimental")

In [5]:
market.connect()
data = market.retrieve("forex")
prices = data.melt(id_vars="date",value_vars=[x for x in data.columns if x != "date"]).rename(columns={"variable":"ticker","value":"adjclose"})
market.disconnect()
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]

In [6]:
prices

,date,ticker,adjclose,year,quarter,week
0,2000-01-01,ANG,1.790000,2000,1,52
1,2000-01-02,ANG,1.790000,2000,1,52
2,2000-01-03,ANG,1.790000,2000,1,1
3,2000-01-04,ANG,1.790000,2000,1,1
4,2000-01-05,ANG,1.790000,2000,1,1
...,...,...,...,...,...,...
1446015,2023-04-11,VES,24.361623,2023,2,15
1446016,2023-04-12,VES,24.442814,2023,2,15
1446017,2023-04-13,VES,24.457308,2023,2,15
1446018,2023-04-14,VES,24.511235,2023,2,15


In [7]:
new_prices = []
for ticker in tqdm(prices["ticker"].unique()):
    ticker_sim = prices[prices["ticker"]==ticker].sort_values("date").copy()
    for i in range(2,5):
        ticker_sim[f"return_{i}"] = (ticker_sim["adjclose"].shift(-i) - ticker_sim["adjclose"].shift(-1)) / ticker_sim["adjclose"].shift(-1)
    new_prices.append(ticker_sim)
price_returns = pd.concat(new_prices)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 170/170 [00:12<00:00, 13.72it/s]


In [8]:
speculation_db.connect()
simulation = speculation_db.retrieve("sim")
speculation_db.disconnect()

In [9]:
classification_db.connect()
classification_simulation = classification_db.retrieve("sim")
classification_db.disconnect()

In [10]:
simulation["week"] = simulation["week"] + 1
classification_simulation["week"] = classification_simulation["week"] + 1
classification_simulation.rename(columns={"prediction":"classification_prediction"},inplace=True)

In [11]:
simulation = price_returns.merge(simulation.drop("adjclose",axis=1),on=["year","week","ticker"],how="left") \
                    .merge(classification_simulation.drop(["adjclose","training_years"],axis=1),on=["year","week","ticker"],how="left")

In [12]:
sim = simulation.dropna()

In [13]:
sim["delta"] = (sim["prediction"] - sim["adjclose"]) / sim["adjclose"]

In [14]:
sim.sort_values("date",inplace=True)
sim

,date,ticker,adjclose,year,quarter,week,return_2,return_3,return_4,y_x,...,d3,rolling14,xgb_prediction_y,xgb_score_y,cat_prediction_y,cat_score_y,tf_prediction_y,tf_score_y,classification_prediction,delta
6946,2019-01-07,ANG,1.775022,2019,1,2,2.512387e-04,0.000223,-1.994137e-04,1.775146,...,-2.135903,1.773449,0.0,0.594588,0.0,0.598471,0.0,0.526271,False,0.619395
1138244,2019-01-07,GEL,2.674962,2019,1,2,-1.200743e-05,-0.000012,9.005574e-06,2.666427,...,-0.043803,5.362997,1.0,0.594588,1.0,0.598471,1.0,0.526271,True,0.554311
602366,2019-01-07,FJD,2.133150,2019,1,2,-7.422545e-03,-0.007000,-4.697813e-03,2.120621,...,-2.152482,2.129914,1.0,0.594588,1.0,0.598471,1.0,0.526271,True,0.544573
610872,2019-01-07,GIP,0.790317,2019,1,2,1.258750e-02,-0.000337,3.846252e-05,0.782835,...,-2.155637,0.777625,0.0,0.594588,0.0,0.598471,1.0,0.526271,False,0.461899
1129738,2019-01-07,CDF,1625.000208,2019,1,2,1.279213e-07,-0.000615,3.862239e-07,1625.714335,...,-6.750914,1618.622264,1.0,0.594588,1.0,0.598471,1.0,0.526271,True,0.011132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467359,2021-12-31,BDT,85.737435,2021,4,52,0.000000e+00,0.001037,2.763670e-04,85.808452,...,-0.741019,85.691401,0.0,0.577780,0.0,0.591180,1.0,0.518821,False,-0.003079
1190369,2021-12-31,UZS,10820.000335,2021,4,52,-7.375231e-08,0.001617,-1.109057e-08,10824.000082,...,-1.406162,10737.124391,0.0,0.577780,0.0,0.591180,1.0,0.518821,False,-0.009575
475865,2021-12-31,BHD,0.376957,2021,4,52,0.000000e+00,0.000096,1.432524e-04,0.377045,...,-0.653962,0.377043,0.0,0.577780,0.0,0.591180,1.0,0.518821,False,0.888714
628973,2021-12-31,GNF,9325.000355,2021,4,52,-6.788203e-08,0.001340,-2.144775e-02,9318.000184,...,-1.383712,9559.616628,1.0,0.577780,1.0,0.591180,1.0,0.518821,True,0.042021


In [15]:
reqs = [0.15]
signals = [0.05,0.1]
values = [True,False]
classifications = [True,False]
ceilings = [True,False]
floors = [True,False]
hedges = [True,False]
parameters = []
training_year = 4
for value in values:
    for classification in classifications:
        for ceiling in ceilings:
            for floor in floors:
                for hedge in hedges:
                    for signal in signals:
                        for req in reqs:
                            parameter = {"value":value
                                         ,"classification":classification
                                         ,"ceiling":ceiling
                                         ,"floor":floor
                                         ,"hedge":hedge
                                         ,"signal":signal
                                         ,"req":req
                                         ,"training_years":training_year}
                            parameters.append(parameter)

In [17]:
len(parameters)

64

In [18]:
parameter = parameters[0]

In [19]:
parameter

{'value': True,
 'classification': True,
 'ceiling': True,
 'floor': True,
 'hedge': True,
 'signal': 0.05,
 'req': 0.15,
 'training_years': 4}

In [20]:
experimental.connect()
experimental.drop("trades")
for parameter in tqdm(parameters):
    current_sim = sim[sim["training_years"]==parameter["training_years"]].reset_index().copy()
    b.experimental_backtest(current_sim,parameter,start_date,end_date,experimental)
experimental.disconnect()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [38:03<00:00, 35.68s/it]
